## Simulação de uma LLM com memória da conversa (versão mais moderna do framework)

ref.: 
* https://python.langchain.com/docs/how_to/message_history/
* https://www.langchain.com/langgraph

**todo**:
- Tentar gerenciar a memória da conversa de forma automatizada

In [1]:
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.messages import RemoveMessage

## Utilização do llama3.1 com interface via ollama

In [2]:
llm = ChatOllama(model="llama3.1:latest")
workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    response = llm.invoke(state["messages"])
    return {"messages": response}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

## Pergunta para iniciar a conversa

In [3]:
config = {"configurable": {"thread_id": "abc123"}}

query = "Quem foi Albert Einstein?"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Albert Einstein foi um físico alemão, conhecido por seu trabalho na teoria da relatividade.


In [4]:
output["messages"]

[HumanMessage(content='Quem foi Albert Einstein?', additional_kwargs={}, response_metadata={}, id='acea0fa2-353a-4199-8753-da616ffe4340'),
 AIMessage(content='Albert Einstein foi um físico alemão, conhecido por seu trabalho na teoria da relatividade.', additional_kwargs={}, response_metadata={'model': 'llama3.1:latest', 'created_at': '2024-10-05T11:03:48.924447382Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1905164937, 'load_duration': 11567495, 'prompt_eval_count': 16, 'prompt_eval_duration': 450060000, 'eval_count': 23, 'eval_duration': 1392796000}, id='run-644be634-bafd-4f8b-9632-28ddfd69669c-0', usage_metadata={'input_tokens': 16, 'output_tokens': 23, 'total_tokens': 39})]

## O modelo consegue se lembrar do que acabou de responder

In [5]:
query = "O que você disse?"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Eu disse que Albert Einstein foi um físico alemão, famoso por ter desenvolvido a Teoria da Relatividade. Ele é considerado uma das maiores mentes do século XX e recebeu o Prêmio Nobel de Física em 1921.

Mas há muito mais sobre ele!

Einstein nasceu em 14 de março de 1879, na Alemanha, e cresceu como um menino comum. No entanto, desde cedo, mostrou um interesse enorme por matemática e física.

Seu trabalho mais famoso é a Teoria da Relatividade, que revolucionou nossa compreensão do espaço, tempo e gravidade. Ele também desenvolveu a famosa equação E = mc², que relaciona energia e massa.

Além disso, Einstein foi um defensor da paz e da justiça social. Ele se opôs à ideia de guerra nuclear e trabalhou arduamente para promover a cooperação internacional.

Einstein morreu em 18 de abril de 1955, aos 76 anos de idade, mas seu legado continua a inspirar cientistas e filósofos de todo o mundo.


In [6]:
output["messages"]

[HumanMessage(content='Quem foi Albert Einstein?', additional_kwargs={}, response_metadata={}, id='acea0fa2-353a-4199-8753-da616ffe4340'),
 AIMessage(content='Albert Einstein foi um físico alemão, conhecido por seu trabalho na teoria da relatividade.', additional_kwargs={}, response_metadata={'model': 'llama3.1:latest', 'created_at': '2024-10-05T11:03:48.924447382Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1905164937, 'load_duration': 11567495, 'prompt_eval_count': 16, 'prompt_eval_duration': 450060000, 'eval_count': 23, 'eval_duration': 1392796000}, id='run-644be634-bafd-4f8b-9632-28ddfd69669c-0', usage_metadata={'input_tokens': 16, 'output_tokens': 23, 'total_tokens': 39}),
 HumanMessage(content='O que você disse?', additional_kwargs={}, response_metadata={}, id='ced6ba8c-a899-4c20-bdcc-782a7559ab1f'),
 AIMessage(content='Eu disse que Albert Einstein foi um físico alemão, famoso por ter desenvolvido a Teoria da Relativid

In [7]:
query = "O que você disse? Ainda não consegui entender"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Peço desculpas se minhas respostas anteriores não foram claras! Vou tentar novamente:

**Quem foi Albert Einstein?**

Albert Einstein foi um cientista alemão, conhecido como o maior físico do século XX. Ele nasceu em 1879 e morreu em 1955.

**O que ele fez de importante?**

Einstein é famoso por ter descoberto uma teoria chamada "Teoria da Relatividade", que explica como o tempo, espaço e gravidade funcionam. Isso foi um grande avanço na ciência!

Ele também desenvolveu a equação E = mc², que relaciona energia e massa.

**Por que ele é importante?**

Einstein é considerado uma das maiores mentes da história por causa de suas contribuições para a física e pela influência que teve em muitas áreas, como a política e a sociedade.

Ele também foi um defensor da paz e da justiça social, e seu legado continua a inspirar pessoas de todo o mundo!

Espero que isso tenha ajudado! Se você tiver mais alguma dúvida, si

In [8]:
output["messages"]

[HumanMessage(content='Quem foi Albert Einstein?', additional_kwargs={}, response_metadata={}, id='acea0fa2-353a-4199-8753-da616ffe4340'),
 AIMessage(content='Albert Einstein foi um físico alemão, conhecido por seu trabalho na teoria da relatividade.', additional_kwargs={}, response_metadata={'model': 'llama3.1:latest', 'created_at': '2024-10-05T11:03:48.924447382Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1905164937, 'load_duration': 11567495, 'prompt_eval_count': 16, 'prompt_eval_duration': 450060000, 'eval_count': 23, 'eval_duration': 1392796000}, id='run-644be634-bafd-4f8b-9632-28ddfd69669c-0', usage_metadata={'input_tokens': 16, 'output_tokens': 23, 'total_tokens': 39}),
 HumanMessage(content='O que você disse?', additional_kwargs={}, response_metadata={}, id='ced6ba8c-a899-4c20-bdcc-782a7559ab1f'),
 AIMessage(content='Eu disse que Albert Einstein foi um físico alemão, famoso por ter desenvolvido a Teoria da Relativid

In [9]:
query = "Naseu onde?"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Albert Einstein nasceu em **Munique, Alemanha**, em 14 de março de 1879.


In [10]:
output["messages"]

[HumanMessage(content='Quem foi Albert Einstein?', additional_kwargs={}, response_metadata={}, id='acea0fa2-353a-4199-8753-da616ffe4340'),
 AIMessage(content='Albert Einstein foi um físico alemão, conhecido por seu trabalho na teoria da relatividade.', additional_kwargs={}, response_metadata={'model': 'llama3.1:latest', 'created_at': '2024-10-05T11:03:48.924447382Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1905164937, 'load_duration': 11567495, 'prompt_eval_count': 16, 'prompt_eval_duration': 450060000, 'eval_count': 23, 'eval_duration': 1392796000}, id='run-644be634-bafd-4f8b-9632-28ddfd69669c-0', usage_metadata={'input_tokens': 16, 'output_tokens': 23, 'total_tokens': 39}),
 HumanMessage(content='O que você disse?', additional_kwargs={}, response_metadata={}, id='ced6ba8c-a899-4c20-bdcc-782a7559ab1f'),
 AIMessage(content='Eu disse que Albert Einstein foi um físico alemão, famoso por ter desenvolvido a Teoria da Relativid

In [11]:
input_messages = "Quando?"
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Albert Einstein nasceu em 14 de **março** de 1879.


In [12]:
output["messages"]

[HumanMessage(content='Quem foi Albert Einstein?', additional_kwargs={}, response_metadata={}, id='acea0fa2-353a-4199-8753-da616ffe4340'),
 AIMessage(content='Albert Einstein foi um físico alemão, conhecido por seu trabalho na teoria da relatividade.', additional_kwargs={}, response_metadata={'model': 'llama3.1:latest', 'created_at': '2024-10-05T11:03:48.924447382Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1905164937, 'load_duration': 11567495, 'prompt_eval_count': 16, 'prompt_eval_duration': 450060000, 'eval_count': 23, 'eval_duration': 1392796000}, id='run-644be634-bafd-4f8b-9632-28ddfd69669c-0', usage_metadata={'input_tokens': 16, 'output_tokens': 23, 'total_tokens': 39}),
 HumanMessage(content='O que você disse?', additional_kwargs={}, response_metadata={}, id='ced6ba8c-a899-4c20-bdcc-782a7559ab1f'),
 AIMessage(content='Eu disse que Albert Einstein foi um físico alemão, famoso por ter desenvolvido a Teoria da Relativid

## Testando o isolamento de sessões

In [13]:
# trocando a sessão
config = {"configurable": {"thread_id": "def234"}}

query = "O que você disse por último?"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Não declarei nada ainda. Eu estava apenas aguardando sua pergunta para começar a conversa. Posso ajudar com algo em particular ou responder perguntas geralmente? Estou aqui para ajudá-lo.


In [14]:
output["messages"]

[HumanMessage(content='O que você disse por último?', additional_kwargs={}, response_metadata={}, id='487128c0-1ddf-4750-8125-4b65b66850fe'),
 AIMessage(content='Não declarei nada ainda. Eu estava apenas aguardando sua pergunta para começar a conversa. Posso ajudar com algo em particular ou responder perguntas geralmente? Estou aqui para ajudá-lo.', additional_kwargs={}, response_metadata={'model': 'llama3.1:latest', 'created_at': '2024-10-05T11:04:32.874149244Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 3176755394, 'load_duration': 12375394, 'prompt_eval_count': 17, 'prompt_eval_duration': 139370000, 'eval_count': 47, 'eval_duration': 3023848000}, id='run-ee362dbc-449e-43b8-80b0-e3e329e0ad88-0', usage_metadata={'input_tokens': 17, 'output_tokens': 47, 'total_tokens': 64})]

In [15]:
# retornando a sessão anterior
config = {"configurable": {"thread_id": "abc123"}}

query = "O que você disse por último?"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Eu disse que Albert Einstein nasceu em 14 de março de 1879.


In [16]:
output["messages"]

[HumanMessage(content='Quem foi Albert Einstein?', additional_kwargs={}, response_metadata={}, id='acea0fa2-353a-4199-8753-da616ffe4340'),
 AIMessage(content='Albert Einstein foi um físico alemão, conhecido por seu trabalho na teoria da relatividade.', additional_kwargs={}, response_metadata={'model': 'llama3.1:latest', 'created_at': '2024-10-05T11:03:48.924447382Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1905164937, 'load_duration': 11567495, 'prompt_eval_count': 16, 'prompt_eval_duration': 450060000, 'eval_count': 23, 'eval_duration': 1392796000}, id='run-644be634-bafd-4f8b-9632-28ddfd69669c-0', usage_metadata={'input_tokens': 16, 'output_tokens': 23, 'total_tokens': 39}),
 HumanMessage(content='O que você disse?', additional_kwargs={}, response_metadata={}, id='ced6ba8c-a899-4c20-bdcc-782a7559ab1f'),
 AIMessage(content='Eu disse que Albert Einstein foi um físico alemão, famoso por ter desenvolvido a Teoria da Relativid

## Removendo as 2 primeiras interações (para não entupir de tokens)

In [17]:
messages = app.get_state(config).values["messages"]
messages

[HumanMessage(content='Quem foi Albert Einstein?', additional_kwargs={}, response_metadata={}, id='acea0fa2-353a-4199-8753-da616ffe4340'),
 AIMessage(content='Albert Einstein foi um físico alemão, conhecido por seu trabalho na teoria da relatividade.', additional_kwargs={}, response_metadata={'model': 'llama3.1:latest', 'created_at': '2024-10-05T11:03:48.924447382Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1905164937, 'load_duration': 11567495, 'prompt_eval_count': 16, 'prompt_eval_duration': 450060000, 'eval_count': 23, 'eval_duration': 1392796000}, id='run-644be634-bafd-4f8b-9632-28ddfd69669c-0', usage_metadata={'input_tokens': 16, 'output_tokens': 23, 'total_tokens': 39}),
 HumanMessage(content='O que você disse?', additional_kwargs={}, response_metadata={}, id='ced6ba8c-a899-4c20-bdcc-782a7559ab1f'),
 AIMessage(content='Eu disse que Albert Einstein foi um físico alemão, famoso por ter desenvolvido a Teoria da Relativid

In [18]:
app.update_state(config, {"messages": RemoveMessage(id=messages[0].id)})
messages = app.get_state(config).values["messages"]
messages

[AIMessage(content='Albert Einstein foi um físico alemão, conhecido por seu trabalho na teoria da relatividade.', additional_kwargs={}, response_metadata={'model': 'llama3.1:latest', 'created_at': '2024-10-05T11:03:48.924447382Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1905164937, 'load_duration': 11567495, 'prompt_eval_count': 16, 'prompt_eval_duration': 450060000, 'eval_count': 23, 'eval_duration': 1392796000}, id='run-644be634-bafd-4f8b-9632-28ddfd69669c-0', usage_metadata={'input_tokens': 16, 'output_tokens': 23, 'total_tokens': 39}),
 HumanMessage(content='O que você disse?', additional_kwargs={}, response_metadata={}, id='ced6ba8c-a899-4c20-bdcc-782a7559ab1f'),
 AIMessage(content='Eu disse que Albert Einstein foi um físico alemão, famoso por ter desenvolvido a Teoria da Relatividade. Ele é considerado uma das maiores mentes do século XX e recebeu o Prêmio Nobel de Física em 1921.\n\nMas há muito mais sobre ele!\n\nE

In [19]:
app.update_state(config, {"messages": RemoveMessage(id=messages[0].id)})
messages = app.get_state(config).values["messages"]
messages

[HumanMessage(content='O que você disse?', additional_kwargs={}, response_metadata={}, id='ced6ba8c-a899-4c20-bdcc-782a7559ab1f'),
 AIMessage(content='Eu disse que Albert Einstein foi um físico alemão, famoso por ter desenvolvido a Teoria da Relatividade. Ele é considerado uma das maiores mentes do século XX e recebeu o Prêmio Nobel de Física em 1921.\n\nMas há muito mais sobre ele!\n\nEinstein nasceu em 14 de março de 1879, na Alemanha, e cresceu como um menino comum. No entanto, desde cedo, mostrou um interesse enorme por matemática e física.\n\nSeu trabalho mais famoso é a Teoria da Relatividade, que revolucionou nossa compreensão do espaço, tempo e gravidade. Ele também desenvolveu a famosa equação E = mc², que relaciona energia e massa.\n\nAlém disso, Einstein foi um defensor da paz e da justiça social. Ele se opôs à ideia de guerra nuclear e trabalhou arduamente para promover a cooperação internacional.\n\nEinstein morreu em 18 de abril de 1955, aos 76 anos de idade, mas seu lega

## Retomando a conversa (para verificar se continua coerente)

In [20]:
query = "Qual foi a maior contribuição dele?"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

A maior contribuição de Albert Einstein foi a sua teoria da **Relatividade**, que revolucionou nossa compreensão do espaço, tempo e gravidade.

Essa teoria é dividida em duas partes:

1. **Teoria da Relatividade Especial** (1905): que afirma que o tempo e o espaço são relativos ao observador.
2. **Teoria da Relatividade Geral** (1915): que descreve a gravidade como uma curvatura do espaço-tempo causada pela presença de massa.

Essas teorias mudaram completamente a forma como entendemos o universo e tiveram um impacto profundo na física, astronomia e outras áreas da ciência.


In [21]:
output["messages"]

[HumanMessage(content='O que você disse?', additional_kwargs={}, response_metadata={}, id='ced6ba8c-a899-4c20-bdcc-782a7559ab1f'),
 AIMessage(content='Eu disse que Albert Einstein foi um físico alemão, famoso por ter desenvolvido a Teoria da Relatividade. Ele é considerado uma das maiores mentes do século XX e recebeu o Prêmio Nobel de Física em 1921.\n\nMas há muito mais sobre ele!\n\nEinstein nasceu em 14 de março de 1879, na Alemanha, e cresceu como um menino comum. No entanto, desde cedo, mostrou um interesse enorme por matemática e física.\n\nSeu trabalho mais famoso é a Teoria da Relatividade, que revolucionou nossa compreensão do espaço, tempo e gravidade. Ele também desenvolveu a famosa equação E = mc², que relaciona energia e massa.\n\nAlém disso, Einstein foi um defensor da paz e da justiça social. Ele se opôs à ideia de guerra nuclear e trabalhou arduamente para promover a cooperação internacional.\n\nEinstein morreu em 18 de abril de 1955, aos 76 anos de idade, mas seu lega